In [1]:
import cv2 as cv
import numpy as np
import math

# Initialize global variables
drawing = False  # True if mouse is pressed
ix, iy = -1, -1  # Initial coordinates of rectangle
rect = None  # Rectangle to be drawn
img = np.ones((500, 500, 3), np.uint8) * 255

def draw_rectangle(event, x, y, flags, params):
    """
    Draw rectangle on mouse click and drag.

    Args:
        event (int): Event code.
        x (int): Mouse x-coordinate on the screen.
        y (int): Mouse y-coordinate on the screen.
        flags (int): Additional information about the event.
        param : User-defined parameter passed to the callback function.
    """
    global ix, iy, drawing, img, rect

    if event == cv.EVENT_LBUTTONDOWN:
        img[:]=255
        if rect is not None:
            # If a rectangle already exists, reset it
            rect = None
        drawing = True
        ix, iy = x, y 

    elif event == cv.EVENT_MOUSEMOVE:
        if drawing:
            img_copy = img.copy()
            cv.rectangle(img_copy, (ix, iy), (x, y), (0, 0, 255), thickness = -1)
#             cv.imshow("Rectangle Window", img_copy)

            if rect is not None:
                cv.rectangle(img_copy, rect, (0, 0, 255), thickness = -1)
                cv.imshow("Rectangle Window", img_copy)
            else:
                cv.imshow("Rectangle Window", img_copy)

    elif event == cv.EVENT_LBUTTONUP:
        drawing = False
        if rect is None:
            # Set the final rectangle coordinates
            rect = (min(ix, x), min(iy, y), abs(x-ix), abs(y-iy))

def translate_rectangle():
    """
    Translate the drawn rectangle based on user input via keyboard.
    """
    global rect, img

    if rect is None:
        print("Please draw a rectangle first.")
    else:
        tx = int(input("Enter translation in x direction: "))
        ty = int(input("Enter translation in y direction: "))
        M = np.float32([[1, 0, tx], [0, 1, ty]])
        translated_img = cv.warpAffine(img, M, (img.shape[1], img.shape[0]))
        rect = (rect[0] + tx, rect[1] + ty, rect[2], rect[3])
        cv.rectangle(translated_img, rect, (0, 0, 255), 2)

def scale_rectangle():
    """
    Scale the drawn rectangle based on user input via keyboard.
    """
    global rect, img

    if rect is None:
        print("Please draw a rectangle first.")
    else:
        scale_factor = float(input("Enter scaling factor (e.g. 0.5 for half size, 2.0 for double size): "))
        center = ((rect[0]+rect[2]//2), (rect[1]+rect[3]//2))
        M = cv.getRotationMatrix2D(center, 0, scale_factor)
        scaled_img = cv.warpAffine(img, M, (img.shape[1], img.shape[0]))
        
        # Update the dimensions of the rectangle
        new_w = int(rect[2] * scale_factor)
        new_h = int(rect[3] * scale_factor)
        rect = (center[0] - new_w//2, center[1] - new_h//2, new_w, new_h)
        # Draw the scaled rectangle on the image
        cv.rectangle(scaled_img, rect, (0, 0, 255), 2)
        cv.imshow("Scaled Rectangle Window", scaled_img)
        
def rotate_rectangle(angle, color):
    """
    Rotate the rectangle on the screen.

    Args:
        angle (int): Angle to rotate the rectangle by (in degrees).
        color (tuple): Color of the rotated rectangle in BGR format.

    """
    global rect, img
    img[:]=255
    if rect is None:
        return
    
    # Get the center point of the rectangle
    cx = rect[0] + rect[2] // 2
    cy = rect[1] + rect[3] // 2
    
    # Define the rotation matrix
    M = cv.getRotationMatrix2D((cx, cy), angle, 1.0)
    
    # Compute the rotated rectangle vertices
    rect_vertices = np.array([[rect[0], rect[1]],
                              [rect[0] + rect[2], rect[1]],
                              [rect[0] + rect[2], rect[1] + rect[3]],
                              [rect[0], rect[1] + rect[3]]],
                             dtype=np.float64)
    
    rotated_vertices = cv.transform(np.array([rect_vertices]), M)[0]

#    Reset the "rect" variable
    rect = None
    # Draw the rotated rectangle in red
    cv.fillConvexPoly(img, np.int32(rotated_vertices), color)
    
    # Compute the bounding box of the rotated rectangle
    x, y, w, h = cv.boundingRect(np.int32(rotated_vertices))
    cv.rectangle(img, (x, y), (x + w, y + h), color, thickness=2)

    # Show the updated image with both rectangles
    cv.imshow("Rectangle Window", img)
    
def reset():
    """
    Reset the image and rectangle.
    """
    global drawing, ix, iy, rect, img

    drawing = False
    ix, iy = -1, -1
    rect = None
    img = np.ones((500, 500, 3), np.uint8) * 255
    cv.imshow("Rectangle Window", img)

# Create a window and set mouse callback function
cv.namedWindow("Rectangle Window")
cv.setMouseCallback("Rectangle Window", draw_rectangle)

while True:
    key = cv.waitKey(1) & 0xFF

    if rect is not None:
        # Draw the final rectangle on the image
        img_copy = img.copy()
        cv.rectangle(img_copy, rect, (0, 0, 255), thickness= -1)
        cv.imshow("Rectangle Window", img_copy)

    else:
        cv.imshow("Rectangle Window", img)

    if key == ord('t'):
        translate_rectangle()
        
    elif key == ord('r'):
        angle = int(input("Enter rotation angle: "))
        color = (0, 255, 0)  # Green color
        rotate_rectangle(angle,color)

    elif key == ord('s'):
        scale_rectangle()
        
    if key == ord('c'):
        reset()

    elif key == ord('q'):
        break

# Release resources and close windows
cv.destroyAllWindows()


Enter rotation angle: 40
